In [21]:
import os
import re
import subprocess
import json
import requests
import time
import sys

In [22]:
with open('config.json', 'r') as file:
    config = json.load(file)

api_key_nist = config['api_key_nist']
api_key_cvedetails = config['api_key_cvedetails']
mongo_uri = config['mongo_uri']

In [23]:
if sys.platform.startswith('win'):
    os_name = "Windows"
elif sys.platform.startswith('linux'):
    os_name = "Linux"

In [24]:
def get_mac_address():
    try:
        # get the mac address
        if os_name == "Windows":
            mac_address = subprocess.check_output("getmac", shell=True)
        if os_name == "Linux":
            mac_address = subprocess.check_output("ifconfig | grep ether", shell=True)

        # extract the mac address
        mac_address = re.search(r"\w\w:\w\w:\w\w:\w\w:\w\w:\w\w", str(mac_address))
        return mac_address.group(0)
    except Exception as e:
        print(f"Error: {e}")
        return None

In [25]:
print(get_mac_address())

Error: 'NoneType' object has no attribute 'group'
None


In [26]:
def get_software_version():
    try:
        # get the software version
        if os_name == "Linux":
            output = os.popen("uname -a").read()
            output_parts = output.split()
            so = output_parts[1]
            so_version = output_parts[2]
        if os_name == "Windows":
            output = os.popen("systeminfo").read()

            # Using regular expressions to extract the OS name and version
            so = re.search(r'OS Name:\s*(.*)', output).group(1)
            so_version = re.search(r'OS Version:\s*(.*)', output).group(1)

        result = json.dumps({
        'so': so,
        'so_version': so_version
        })

        return result
    
    except Exception as e:
        print(f"Error: {e}")
        return None

In [28]:
print(get_software_version())

{"so": "Microsoft Windows 11 Pro", "so_version": "10.0.22631 N/A Build 22631"}


In [29]:
def get_software_version():
    try:
        # get the software version
        if os_name == "Linux":
            output = os.popen("uname -a").read()
            output_parts = output.split()
            so = output_parts[1]
            so_version = output_parts[2]
        if os_name == "Windows":
            output = os.popen("systeminfo").read()

            # Using regular expressions to extract the OS name and version
            so = re.search(r'OS Name:\s*(.*)', output).group(1)
            so_version = re.search(r'OS Version:\s*(.*)', output).group(1)

        result = json.dumps({
        'so': so,
        'so_version': so_version
        })

        return result
    
    except Exception as e:
        print(f"Error: {e}")
        return None

In [30]:
def GetSoftwareList():
    if os_name == "Windows":
        # Execute the command to get the list of installed software (Windows case)
        proc = subprocess.Popen("wmic product get name", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    elif os_name == "Linux":
        # Execute the command to get the list of installed software (Linux case)
        proc = subprocess.Popen("dpkg -l | awk '{print $2}' | tail -n +6", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Read the command output
    stdout, stderr = proc.communicate()

    # To list, split by product name
    softwareList = stdout.splitlines()

    return softwareList

In [31]:
print(GetSoftwareList())

['Name                                                                           ', '', 'Python 3.11.7 Documentation (64-bit)                                           ', '', 'Microsoft Teams Meeting Add-in for Microsoft Office                            ', '', 'Python 3.12.2 Standard Library (64-bit)                                        ', '', 'Python 3.11.7 Add to Path (64-bit)                                             ', '', 'Python 3.11.7 Executables (64-bit)                                             ', '', 'Python 3.11.7 Tcl/Tk Support (64-bit)                                          ', '', 'Python 3.11.7 Standard Library (64-bit)                                        ', '', 'Python 3.11.7 Utility Scripts (64-bit)                                         ', '', 'Python 3.11.7 pip Bootstrap (64-bit)                                           ', '', 'Python 3.12.2 Executables (64-bit)                                             ', '', 'Python 3.11.7 Development Libraries (64-b

In [10]:
def GetVulnerabilityInfo(vulnId):
    """
    Obtain the info provided by services.nvd.nist.gov about the vulnerability, then store the relevant info in a JSON
    
    Args: 
        vulnId: That's the vulnerability identifier for the endpoint request.

    Returns:
        vulnInfo: JSON that contains relevant info about the vulnerability.
    """
    print('VulnName a pasar = '+ vulnId) 
    endPoint = f'https://services.nvd.nist.gov/rest/json/cves/2.0?cveId={vulnId}'
    headers = {
        'apiKey': api_key_nist
    }
    infoVuln = requests.get(endPoint,headers = headers)
    try:
        infoVulnJSON = infoVuln.json()
    except ValueError:
        print('Error, respuesta recibida fue: ')
        print(infoVuln.json())
        return
    infoVulnJSON = infoVulnJSON['vulnerabilities'][0]['cve']
    cvssMetricInfo = {}
    if 'cvssMetricV31' in infoVulnJSON['metrics']:
        cvssMetricInfo = RestructureCvssMetrics(infoVulnJSON['metrics']['cvssMetricV31'],31)
    elif 'cvssMetricV30' in infoVulnJSON['metrics']:
        cvssMetricInfo = RestructureCvssMetrics(infoVulnJSON['metrics']['cvssMetricV30'],30)
    elif 'cvssMetricV2' in infoVulnJSON['metrics']:
        cvssMetricInfo = RestructureCvssMetrics(infoVulnJSON['metrics']['cvssMetricV2'],2)
    
    vulnInfo = {
        "CVE_ID": vulnId,
        "description": infoVulnJSON['descriptions'][0]['value'],
        "vulnStatus": infoVulnJSON['vulnStatus'],
        "metrics": cvssMetricInfo,
    }
    return vulnInfo

In [11]:
def RestructureCvssMetrics(infoVulnJSON, metricVersion):
    """
    Store in a JSON (cvssMetricInfo) the organized data about cvss metrics
    
    Args: 
        infoVulnJSON: JSON with data still unorganized 
        metricVersion: especifies the cvss version which was calculated the metrics

    Returns:
        JSON: organized data about cvss metrics which contains:
            baseSeverity - baseScore - vertorString and its details - exploitabilityScore - impactScore - type - version cvss Metrics
    """
    cvssMetricInfo = {}
    cvssMetricInfo['baseScore'] = infoVulnJSON[0]['cvssData']['baseScore']
    # Depends cvss version used for metrics the baseSeverity is in diferent JSON's place
    if metricVersion == 31 or metricVersion == 30:
        cvssMetricInfo['baseSeverity'] = infoVulnJSON[0]['cvssData']['baseSeverity']
    else:
        cvssMetricInfo['baseSeverity'] = infoVulnJSON[0]['baseSeverity']
    # Remaining cvss metrics is equally located in any version
    cvssMetricInfo['exploitabilityScore'] = infoVulnJSON[0]['exploitabilityScore']
    cvssMetricInfo['impactScore'] = infoVulnJSON[0]['impactScore']
    cvssMetricInfo['type'] = infoVulnJSON[0]['type']
    cvssMetricInfo['cvssDetailedData'] = infoVulnJSON[0]['cvssData']
    # Deleting repeated info
    del cvssMetricInfo['cvssDetailedData']['baseScore']
    if metricVersion == 31 or metricVersion == 30:
        del cvssMetricInfo['cvssDetailedData']['baseSeverity']
    # Return final restructured JSON
    return cvssMetricInfo

In [12]:
def GetCVEPerSoftware(productName):
    url = f"https://www.cvedetails.com/api/v1/vulnerability/search?outputFormat=json&productName={productName}&pageNumber=1&resultsPerPage=10"
    
    headers = {
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0',
      'Authorization': f'Bearer {api_key_cvedetails}'
    }
    tryRequest = True
    while(tryRequest):   
        session = requests.Session()
        response = session.get(url, headers=headers)
        responseJSON = response.json()
        if "errors" in responseJSON:
            print("time exceded waiting 1 minute")
            time.sleep(60)
        else:
            print("cveList request of " + productName + " done")
            tryRequest = False
    cveList = []
    if len(responseJSON['results']) < 1:
        return "No vulnerabilities registered"
    for cve in responseJSON['results']:
        cveInfo = GetVulnerabilityInfo(cve['cveId'])
        cveAttackDetails = {k:v for k, v in cve.items() if k.startswith('is')}
        cveInfo['AttackDetails'] = cveAttackDetails
        cveList.append(cveInfo)

    return cveList

In [19]:
def InsertInAtlas(results):
    """
    Insert the results in the Atlas MongoDB
    Args: results: List of JSONs with the info to insert
    """
    
    client = pymongo.MongoClient(mongo_uri)
    
    # Selecciona la base de datos
    db = client["DataScience"]
    
    # Selecciona la colección
    coleccion = db["SoftwareCVE"]
    
    coleccion.insert_many(results)
    print("Correctly Inserted")